# Pretrained Model, regression (target MEM-Score), Elastic Net


In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'mean')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [4]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,0.059918,-0.038175,0.013602,-0.007439,0.072988,0.035254,0.005983,0.021569,0.538193,0.359326,0.409272,0.099061,0.155599,0.219386,0.161535,0.621305,0.430063,0.333243,0.092059,0.213101,0.191816,0.532068,0.174604,0.106723,0.326073,0.221852,0.322626,0.020247,0.159319,0.150326,0.175592,0.081184,0.105608,0.355734,0.317396,0.349668
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.10496,-0.027038,0.020629,-0.002622,0.019785,-0.023587,0.012025,0.052178,0.507799,0.359254,0.361147,0.049819,0.130366,-0.137414,0.04217,0.671531,0.473377,0.367554,0.140397,-0.002806,0.095852,0.459918,0.188397,0.030068,0.023321,0.034736,0.384345,0.135988,0.102555,0.07141,0.322519,-0.043426,0.090363,0.192952,0.105353,0.240881
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.07133,0.005331,0.012895,0.026959,0.023896,0.026039,0.033056,0.021831,0.199761,0.074975,0.071457,-0.01712,0.052231,0.00726,0.049056,0.303302,0.190495,0.17687,0.03085,0.020871,-0.008662,0.266599,0.142112,0.027551,0.117675,-0.00927,0.203192,-0.014111,0.051064,-0.016667,0.145192,0.057935,0.06542,0.152252,0.080831,0.19634
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,0.188084,0.052211,-0.111568,-0.09,0.029541,0.036005,0.035995,0.083385,0.482514,0.024467,-0.107833,0.03448,0.060489,-0.116487,0.093149,0.313839,0.182987,0.212667,-0.025838,-0.009318,-0.052479,0.246719,0.081127,-0.046371,0.104513,-0.019481,0.277193,-0.021194,0.038132,-0.001597,0.12639,-0.001889,0.064197,0.17999,0.118052,0.227498
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.132788,0.04202,0.09332,0.060415,0.066898,0.022056,0.051668,0.080887,0.205802,-0.009584,0.094284,0.054277,0.037151,0.15134,0.174541,0.238345,0.136062,0.16104,-0.017533,0.086181,0.09898,0.252812,0.137191,-0.008658,0.242225,0.125819,0.191427,-0.021946,0.161507,0.138011,0.1411,0.009321,0.092701,0.30901,0.235893,0.37739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,0.136389,-0.023649,-0.002348,-0.07102,0.079082,0.033848,0.001036,0.069726,0.545994,0.194554,0.221084,0.132955,-0.041355,0.089987,0.066649,0.27206,0.221954,0.15329,-0.021906,0.093227,0.013773,0.247335,0.072557,-0.043635,0.122286,-0.007813,0.180578,-0.003443,0.078697,0.061495,0.229756,0.035669,0.053695,0.155652,0.11343,0.156167
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,0.178495,-0.096916,0.058761,-0.039244,0.122438,0.031171,0.042034,0.026155,0.290446,-0.056688,-0.056182,-0.144704,0.02513,-0.05151,-0.009278,0.225529,0.038229,0.20374,-0.01182,-0.006614,0.016697,0.253735,0.013468,-0.005859,0.067822,-0.058574,0.410099,-0.045411,0.088022,0.08374,0.220006,0.010572,0.065844,0.178566,0.066199,0.169795
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.097265,-0.026466,-0.028496,-0.019094,0.018134,0.067899,0.024621,0.035075,0.178263,0.056536,0.053847,-0.008416,0.00087,-0.012427,0.012002,0.161124,0.091493,0.08265,0.008871,-0.021607,-0.041121,0.147687,0.024451,-0.011598,0.081368,0.016668,0.137266,0.002639,-0.02314,-0.028287,0.112552,-0.016535,-0.012336,0.127273,0.094503,0.102097
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.131525,-0.043982,0.079655,-0.051607,0.117778,0.101683,0.019386,0.078948,0.163418,-0.094566,0.056415,-0.041263,-0.068523,0.020028,-0.034854,0.430648,0.096152,0.308294,0.093714,-0.058784,-0.10513,0.167568,0.071621,-0.072117,0.017321,-0.143462,0.324986,0.095204,0.018929,-0.069679,0.194042,0.028932,0.10987,0.101379,0.069668,0.249134


In [5]:
classification = False
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "prmdiag", "Apoe", "IDs"]
target = "MEM_score"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [6]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
721,0.973914,-0.500490,0.690935,-0.851425,1.378341,-0.294586,-0.409307,0.695415,0.401510,-2.228484,-0.573307,-3.097717,-0.898754,-1.151178,-0.328843,-0.528070,-1.359650,-0.226142,-1.818475,-1.270575,-2.065530,-1.842311,-1.236392,-1.634027,-0.583783,1.588847,-0.887716,-0.457866,0.174673,-2.612726,-1.493556,-0.450313,-0.412860,-0.660690,-0.002377,1.579485,2.153478,1.833252,-0.292275,2.047959
335,0.281476,-0.174161,0.690935,0.819703,-0.517097,-0.386903,-1.623338,0.376601,-0.655489,0.134835,1.364199,0.231395,-1.192758,0.498229,0.093354,0.400220,-0.044931,-0.351140,-0.432820,1.198496,0.138635,-0.243805,-1.234296,-2.327094,-2.250624,0.278554,0.359227,0.379417,0.127891,-0.265363,-0.367121,-1.354149,-0.096384,-0.793340,1.394470,0.177241,0.013518,1.626623,0.823396,0.044825
590,0.19615,-0.337326,-1.041167,0.151252,-0.636484,-0.191696,0.120593,1.131089,-0.377616,-0.346518,0.597784,0.235161,-0.652024,1.095122,-0.362313,0.500397,0.407238,-1.559100,-0.395032,1.282448,1.103472,-0.778556,1.449680,-1.378961,-1.155275,0.932992,0.159841,0.641547,-0.173127,-0.584591,-0.435720,-0.471218,0.264839,-0.571950,1.169965,-0.414629,0.151628,0.791817,0.688070,1.833569
7,-1.815235,0.641659,0.690935,-0.851425,2.103472,1.772682,0.932929,-0.096382,2.842598,4.251594,1.492703,0.191734,-0.823662,1.425292,1.786037,1.800053,1.611198,1.965026,-0.106917,0.656355,2.343357,-0.350356,0.558106,1.744113,0.295958,3.282943,1.012429,-1.314214,0.797248,-0.597318,-0.465521,1.297234,1.492544,-0.024738,3.785186,-0.099435,-0.910165,-0.452906,-0.308142,-1.409410
567,0.676812,-0.174161,0.690935,-0.517199,0.453356,1.425160,1.221316,1.140397,-0.105968,0.129335,0.427604,0.495274,0.312952,-0.582825,0.301611,1.495428,-0.310755,1.529281,1.224869,-0.773694,-0.668559,-1.229722,-0.162767,-0.262323,-0.557069,-0.083097,-0.392553,0.645771,0.405041,0.878796,-0.277369,0.382228,1.301599,0.181829,-0.627678,0.879461,0.394573,0.513536,0.402065,0.606566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.87983,-0.337326,-1.041167,0.151252,-1.091441,-0.862778,-0.038549,-0.354841,-0.305506,-1.628013,-1.568055,-1.874691,0.002805,-0.919014,0.203128,-0.289152,-0.769100,0.988427,0.061578,-0.461341,0.173037,0.071747,-0.196589,1.369244,1.218185,-0.872062,0.061565,-0.028000,0.070808,0.338541,-0.432435,-0.042948,-0.226854,0.158231,-0.876723,-0.059876,-0.765166,-1.816493,-0.378514,-0.588319
793,-0.39345,0.967987,-1.041167,-0.517199,0.278385,-2.107574,-0.200458,-0.647246,0.233863,-1.283337,-0.469742,-0.770151,1.482162,1.277905,0.849104,0.627659,0.754490,0.939704,0.033996,0.482845,0.293530,0.696906,1.277916,1.982984,2.339160,0.153929,0.769114,-0.225730,0.499522,0.740758,0.073368,1.546833,1.124956,1.294748,2.636379,0.600767,0.051546,-0.417629,1.567315,1.889178
11,1.320656,-1.153146,0.690935,-0.851425,0.303645,1.612388,1.221892,2.294116,-0.119928,0.913584,0.698829,0.522887,-0.611561,-0.172122,-0.196544,0.412306,0.637312,1.132389,1.677233,-0.810482,-0.467811,0.231788,-0.332552,0.349230,0.171715,-0.275952,0.304915,0.579634,0.536203,0.076237,-0.072758,-0.987768,-0.409173,-0.743769,-0.043099,0.746617,0.490791,-0.320369,0.880301,2.185673
317,0.627507,-0.174161,-1.041167,1.488154,0.931078,-0.620579,-0.770250,0.328658,0.180711,-1.008811,1.670059,1.324899,1.676528,0.607396,2.167446,0.791620,-0.570319,0.372973,1.110138,0.867616,1.663583,1.765034,0.439670,1.664007,1.647067,1.460851,1.477543,-0.159822,0.649352,1.763744,1.867694,0.930933,1.620056,1.116473,-0.382143,-0.689684,-0.590766,0.146822,0.813725,-0.069224


# Train Model

In [7]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, classification = False, 
                          n_alphas_linreg = 10, 
                          l1_ratios_linreg= np.linspace(0.01,  1, 11).tolist(),
                          cv_linreg = 5)               

In [9]:
predictions = model.predict(Xtest)
mean_squared_error(ytest, predictions)

0.6311098539360094

In [10]:
r2_score(ytest, predictions)

0.3554127041330113

In [11]:
model.l1_ratio_

0.01

In [12]:
model.alpha_

0.08505588344771099

# Save Model

In [13]:
import pickle

In [14]:
filename ='pretrained_elnet_regr_agg_mean.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [ ]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_regr_agg_mean.sav", 'rb'))